In [12]:
import pandas as pd
# change week
week = 17
test = pd.read_csv(f'Week{week}Test.csv')

# Adjust team stats to accomodate for bye weeks
incrementList = ['PF', 'PA', 'Yds_tot_Off', 'Ply_Off', 'TO_Off', 'FL_Off', '1stD_tot_Off', '1stD_Pss_Off',
                        '1stD_Rsh_Off', 'Pen_Off', 'Yds_Pny_Off', '1stPy_Off', 'G_Pss_Off', 'Cmp_Off', 'Att_Pss_Off', 
                        'Yds_Pss_Off', 'TD_Pss_Off', 'Int_Off', 'Lng_Pss', 'Sk_Off', 'Yds.1_Off', 'G_Rsh_Off', 'Att_Rsh_Off',
                        'Yds_Rsh_Off', 'TD_Rsh_Off', 'Lng_Rsh', 'Fmb', 'G_Off', 'RshTD_Off', 'RecTD_Off', 'AllTD_Off',
                        'XPM_Off', 'XPA_Off', 'FGM_Off', 'FGA_Off', 'Pts_Off', 'Yds_tot_Def', 'Ply_Def', 'TO_Def', 'FL_Def',
                        '1stD_tot_Def', '1stD_Pss_Def', '1stD_Rsh_Def', 'Pen_Def', 'Yds_Pny_Def', '1stPy_Def', 'G_Pss_Def',
                        'Cmp_Def', 'Att_Pss_Def', 'Yds_Pss_Def', 'TD_Pss_Def', 'Int_Def', 'PD_y', 'Sk_Def', 'Yds.1_Def',
                        'QBHits', 'TFL', 'G_Rsh_Def', 'Att_Rsh_Def', 'Yds_Rsh_Def', 'TD_Rsh_Def', 'RshTD_Def', 'RecTD_Def',
                        'AllTD_Def', 'XPM_Def', 'XPA_Def', 'FGM_Def', 'FGA_Def', 'Pts_Def']
location = ['_home', '_away']
for loc in location:
    for col in incrementList:
        change = col + loc
        test.loc[test['W'+loc] + test['L'+loc]== week-2, change] += (test[change]/(week-2))
for loc in location:
    for col in incrementList:
        change = col + loc
        test.loc[test['W'+loc] + test['L'+loc]== week-1, change] = 17*(test[change]/(week-1))

# Helper function to determine if our prediction beat Vegas
def determine_beat_odds(row):
    if row['pred'] > row['Vegas'] and row['actual'] > row['Vegas']:
        return True
    elif row['pred'] < row['Vegas'] and row['actual'] < row['Vegas']:
        return True
    elif row['pred'] == row['Vegas'] or row['actual'] == row['Vegas']:
        return True
    else:
        return False

# import trained model from directory
from tensorflow.keras.models import load_model
loaded_model = load_model('nflfinal')
model = loaded_model

test = test.drop(columns=['Week', 'Time', 'Away', 'Tm_away', 'Home', 'Tm_home', 'D2P_Def_away','D2P_Def_home' ])
newCol = {'L_away': 'losses_away', 'L_home': 'losses_home', 'W-L%_away': 'wlp_away',
          'W-L%_home': 'wlp_home', 'W_away': 'wins_away', 'W_home': 'wins_home'}
test = test.rename(columns=newCol)
pred = test
predictions = model.predict(pred)
count = 0
iterate = pd.read_csv(f'Week{week}Test.csv')

teams =  iterate['Home'].unique()
teams = [team[:-5] for team in teams]

init = {'Name': teams,
        'Wins': [0]*32,
        'Losses': [0]*32}

simulate = pd.DataFrame(init)
count = 0
weeklyLines = []
for index, row in iterate.iterrows():
    #print(f"{row['Week']} {row['Away']} vs Home: {row['Home']}: {predictions[count]}")
    home = (row['Home'][:-5])
    away = (row['Away'][:-5])
    spread = (predictions[count][0])
    formattedSpread = str(abs(spread))
    formattedSpread = formattedSpread[:3] if formattedSpread[1] == '.' else formattedSpread[:4]

    if (spread > 0):
        #print(f"Week {row['Week']}: {away} +{formattedSpread}, {home} -{formattedSpread}")
        simulate.loc[simulate['Name'] == home, 'Wins'] += 1
        simulate.loc[simulate['Name'] == away, 'Losses'] += 1
        if (row['Week'] == week):
            weeklyLines.append([week, away,f"+{formattedSpread}", home, f"-{formattedSpread}" ] )
    else:
        #print(f"Week {row['Week']}: {away} -{formattedSpread}, {home} +{formattedSpread}")
        simulate.loc[simulate['Name'] == away, 'Wins'] += 1
        simulate.loc[simulate['Name'] == home, 'Losses'] += 1
        if (row['Week'] == week):
            weeklyLines.append([week, away,f"-{formattedSpread}", home, f"+{formattedSpread}" ] )

    count+=1 
#print(simulate)
# ^uncomment to simulate entire regular season
lines = pd.DataFrame(weeklyLines, columns=['Week', 'Away', 'Away_spread', 'Home', "Home_spread"])
print(lines)
if week == 10: 
    results = [-3, 4, 2, -4, 3, -8, -14, 31, -2, 3, -32, -3, -4, 2] #week 10
    vegas = [-3, 1.5, -5.5, -3, -5.5, 3, -2.5, 3, 2, 2, -17.5, -6, 1, -7.5]
elif week == 11: 
    results = [-14, -5, -5, 23, 12, -20, -3, -7, -3, -13, -26, -1, -1, 4] #week 11
    vegas = [-4, -8, -5.5, 11, -7.5, -6.5, -2.5, -14, 3, -13.5, -7.5, -2.5, -2.5, -2]
elif week == 12: 
    results = [7, -35, 18, 21, -7, 3, -9, -3, 6, -7, -17, 23, -3, 14, 10, -100] #week 12
    vegas = [-8.5, -13, 7, 9, -3.5, 1, 2, 4.5, 2, -2.5, -2, 3, -2.5, 9, 3, -3]
elif week == 13: 
    results = [-6,5, -3, 3, 14, 5, 30, 6, -5, -17, 23, -8, 3] #week 13
    vegas = [-9.5, 2, -3.5, 1.5, -6, 4, 8.5, 4.5, -3, -3.5, 3, 6, -10]
elif week == 14: 
    results = [3, -22, -20, -15, -24, -4, -6, 4, 3, -12, 3, 17, 20, -2, 1] #week 14
    vegas = [-5, -5, -3, 3, 3, -2, -7.5, -1.5, 3, -15, -2, -3, -3.5, 5.5, -13.5]
elif week == 15: 
    results = [-42, -2, -3, -25, 3, -3, -18, -30, -17, 14, 16, -8, -21, -3, 16, 10] #week 15
    vegas = [-3, 2.5, -3, -5.5, -3, -3, -6, -7, -1, -4, 12.5, -7, -2.5, 5, 4, 10]
elif week == 16: 
    results = [-8, -23, 2, 14, 19, 6, 3, 3, -2, -18, -11, -2, 3, 6, -8, 14] #week 16
    vegas = [-4, 3, 12.5, 3, -2.5, 2.5, 3.5, 3, -3, 2.5, -4.5, -1.5, -7, -11, -14, -6.5]
elif week == 17: 
    results = [-17, -1, -20, -26, 4, -37, 20, -6, -23, -3, 1, 17, 7, -8, -7, 23] #week 17
    vegas = [-7, -4.5, -3, -7, -10.5, -4, -3, -15, -3.5, -3, 6, 13.5, -3.5, -7, -5.5, -2]
elif week == 18: 
    results = [-31, 7, -8, -17, 28, -13, 4, -8, 1, -10, 14, -17, 7, 1, 1, 9] #week 18
    vegas = [-3, 2.5, -2.5, -7, 13, -3.5, 1.5, 3.5, -3.5, -3.5, -2.5, 5, 3, -5.5, 3, 4.5]
elif week == 19: 
    results = [-31, -14, -1, -19, 16, -23] #Wildcard
    vegas = [2.5,-10, -3, -4.5, -7, 3]
elif week == 20: 
    results = [-24, -3, -8, 3] #Divisional
    vegas = [-10, -10.5, -6, -2.5]
elif week == 21: 
    results = [7, -3] #Championship
    vegas = [-3.5, -7]
elif week == 22: 
    results = [3] #Super Bowl
    vegas = [-1]
# source for vegas line -> https://www.vegasinsider.com/nfl/odds/las-vegas/
# apply logic to measure performance
lines['actual'] = results
lines['Vegas'] = vegas
lines['Home_spread'] = lines['Home_spread'].str.replace('+', '')
lines['pred'] = (lines['Home_spread'].astype(float))
lines['oneScore'] = abs(round(lines['Home_spread'].astype(float)) - lines['actual']) <= 8
lines['predWinner'] = round(lines['Home_spread'].astype(float)* lines['actual']) >= 0
lines['beatOdds'] = lines.apply(determine_beat_odds, axis=1)
print("1 score: " +str(lines['oneScore'].sum())+"/"+str(len(lines)))
print("Correct Winner: " +str(lines['predWinner'].sum())+"/"+str(len(lines)))
print("Beat Vegas: " +str(lines['beatOdds'].sum())+"/"+str(len(lines)))
#print (lines)
# ^uncomment to print metric table

9/9 [==============================] - 0s 938us/step
    Week                  Away Away_spread                   Home Home_spread
0     17         New York Jets       +12.2       Cleveland Browns       -12.2
1     17         Detroit Lions        +3.3         Dallas Cowboys        -3.3
2     17       Atlanta Falcons        +1.3          Chicago Bears        -1.3
3     17     Carolina Panthers        +9.9   Jacksonville Jaguars        -9.9
4     17     Arizona Cardinals        +8.1    Philadelphia Eagles        -8.1
5     17        Miami Dolphins        -5.2       Baltimore Ravens        +5.2
6     17    New Orleans Saints        +3.8   Tampa Bay Buccaneers        -3.8
7     17  New England Patriots       +13.6          Buffalo Bills       -13.6
8     17      Tennessee Titans       +10.4         Houston Texans       -10.4
9     17     Las Vegas Raiders       +13.3     Indianapolis Colts       -13.3
10    17      Los Angeles Rams        +4.9        New York Giants        -4.9
11    17   

/var/folders/c3/4jwj2hy164s7ghr3rk_vnv9c0000gn/T/ipykernel_7680/509095694.py:130: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  lines['Home_spread'] = lines['Home_spread'].str.replace('+', '')
